In [1]:
%load_ext autoreload

import time
from extraction_package import extraction_pipeline

In [2]:
filenames = ["Nickel Royale Ni 12-2003.pdf","NorthMet Cu Ni PGE 10-2012.pdf"]
url_list = ["https://w3id.org/usgs/z/4530692/5PGHWDVA", "https://w3id.org/usgs/z/4530692/64679KK8"]
commodity = "Nickel"
commodity_sign = "Ni"
file_path = "./reports/nickel/"
output_path = "./extracted/"

In [3]:
%autoreload
#multiprocessing test

t = time.time()
file_extractions = extraction_pipeline.document_parallel_extract(
    file_path,
    filenames,
    url_list,
    commodity,
    commodity_sign,
    output_path
        )

print(f'parallel: {time.time()-t}')

Running the parallelization method with 2 files
Working on file: Nickel Royale Ni 12-2003.pdf url: https://w3id.org/usgs/z/4530692/5PGHWDVA commodity of interest is Nickel
Working on file: NorthMet Cu Ni PGE 10-2012.pdf url: https://w3id.org/usgs/z/4530692/64679KK8 commodity of interest is Nickel
file_key 64679KK8 Title: NI 43-101 Technical Report for the NorthMet Project in Minnesota, United States dated October 2012
file_key 5PGHWDVA Title: NI 43-101 Technical Report for the Nickel Royale Project in North America dated 2003
Created an Assistant
Created an Assistant
Current run id = run_hEQTuExoFoOoHGSUlVzNkrVx thread_id = thread_Ihe1uAvnnWP46lC8gRNIW6Qg
Checking run status: in_progress
Current run id = run_sKPX60tWUVTB6BsXsfEGHU5p thread_id = thread_qfLuX3qj1x1gIg5VQhcy3LaR
Checking run status: in_progress
Run is completed. Printing the entire thread now in sequential order 

Most run run_hEQTuExoFoOoHGSUlVzNkrVx response: YES 
Response: YES
File was correctly uploaded
Creating the t

Most run run_e53wKQfqOLrxL2Tc4mJbmSnY response: Based on the information extracted from the report, we have the following entries for the inferred nickel resource estimate data:

```json
{
    "extractions": [
        {
            "category": "Inferred",
            "zone": "Total",
            "nickel Cut-Off": "",
            "nickel Cut-Off Unit": "",
            "nickel Tonnage": "230",
            "nickel Tonnage Unit": "million short tons",
            "nickel Grade Percent": "0.08",
            "Contained_metal": "",
            "Table": "Table 15-1 Updated Reserve Estimate – September 2007"
        },
        {
            "category": "Inferred",
            "zone": "",
            "nickel Cut-Off": "US$7.42 NMV",
            "nickel Cut-Off Unit": "",
            "nickel Tonnage": "",
            "nickel Tonnage Unit": "",
            "nickel Grade Percent": "",
            "Contained_metal": "",
            "Table": "Table 6-2 NorthMet Historical Resource Estimate"
        }

AttributeError: 'NoneType' object has no attribute 'lower'